### 13. Модель BERT и GPT

**Задание**<br>
С помощью бертов решить задачу классификации


**Настроим окружение**

In [1]:
%%capture
!wget https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json
!pip install corus razdel
!pip install datasets transformers seqeval

In [2]:
import pandas as pd
import numpy as np
import torch
import logging

from corus import load_rudrec
from razdel import tokenize
from datasets import load_dataset, load_metric
from datasets import Dataset, DatasetDict
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from transformers import AutoTokenizer
from transformers import pipeline
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from transformers.trainer import logger as noisy_logger

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


В этом блокноте мы дообучаем модель на задаче классификации отдельных слов, а именно, распознавание именованных сущностей (aka named entity recognition, aka NER). Мы возьмём датасет медицинских сущностей, но в целом пайплайн подходит для любой задачи на выделение сущностей в тексте. 

Для ускорения обучения возьмём маленький BERT для русского языка [rubert-tiny](https://huggingface.co/cointegrated/rubert-tiny).


In [3]:
model_checkpoint = "cointegrated/rubert-tiny"
batch_size = 16

**Загрузим данные**

Для обучения мы возьмём [Russian Drug Reaction Corpus](https://github.com/cimm-kzn/RuDReC): размеченный корпус русскоязычных отзывов на лекарства. 

In [4]:
drugs = list(load_rudrec('rudrec_annotated.json'))
print(len(drugs))

4809


**Пример документа:**

In [5]:
drugs[101]

RuDReCRecord(
    file_name='172744.tsv',
    text='нам прописали, так мой ребенок сыпью покрылся, глаза опухли, сверху и снизу на веках высыпала сыпь, ( 8 месяцев сыну)А от виферона такого не было... У кого ещё такие побочки, отзовитесь!1 Чем спасались?\n',
    sentence_id=0,
    entities=[RuDReCEntity(
         entity_id='*[0]_se',
         entity_text='виферона',
         entity_type='Drugform',
         start=122,
         end=130,
         concept_id='C0021735',
         concept_name=nan
     ),
     RuDReCEntity(
         entity_id='*[1]',
         entity_text='сыпью покрылся',
         entity_type='ADR',
         start=31,
         end=45,
         concept_id='C0015230',
         concept_name=nan
     ),
     RuDReCEntity(
         entity_id='*[2]',
         entity_text='глаза опухли',
         entity_type='ADR',
         start=47,
         end=59,
         concept_id='C4760994',
         concept_name=nan
     ),
     RuDReCEntity(
         entity_id='*[3]',
         entity_text

Посмотрим, какие сущности есть: лекарства, форма лекарств, класс лекарств, показания к применению, побочки, и прочие болезни/симптомы.

https://arxiv.org/abs/2004.03659

* **DRUGNAME** Mentions of the brand name of a drug or product
ingredients/active compounds.
* **DRUGCLASS** Mentions of drug classes such as anti-inflammatory or
cardiovascular.
* **DRUGFORM** Mentions of routes of administration such as tablet
or liquid that describe the physical form in which
medication will be delivered into patient’s organism.
* **DI** Any indication/symptom that specifies the reason for
taking/prescribing the drug.
* **ADR** Mentions of untoward medical events that occur as a
consequence of drug intake and are not associated with
treated symptoms.
* **FINDING** Any DI or ADR that was not directly experienced by the
reporting patient or his/her family members, or related to
medical history/drug label, or any disease entities if the
annotator is not clear about type

In [6]:
type2text = defaultdict(Counter)
ents = Counter()
for item in drugs:
    for e in item.entities:
        ents[e.entity_type] += 1
        type2text[e.entity_type][e.entity_text] += 1

for k, v in ents.most_common():
    print(k, v)
    print(type2text[k].most_common(3))

DI 1401
[('простуды', 64), ('ОРВИ', 47), ('профилактики', 42)]
Drugname 1043
[('Виферон', 33), ('Анаферон', 25), ('Циклоферон', 24)]
Drugform 836
[('таблетки', 154), ('таблеток', 79), ('свечи', 63)]
ADR 720
[('аллергия', 16), ('слабость', 13), ('диарея', 12)]
Drugclass 330
[('противовирусный', 21), ('противовирусное', 18), ('противовирусных', 13)]
Finding 236
[('аллергии', 12), ('температуры', 6), ('сонливости', 5)]


In [7]:
drugs[101].text

'нам прописали, так мой ребенок сыпью покрылся, глаза опухли, сверху и снизу на веках высыпала сыпь, ( 8 месяцев сыну)А от виферона такого не было... У кого ещё такие побочки, отзовитесь!1 Чем спасались?\n'

Напишем функцию, перекладывающую разметку сущностей на уровень слов. Будем использовать [IOB](https://en.wikipedia.org/wiki/Inside–outside–beginning_(tagging))-нотацию, чтобы разделять несколько сущностей одного типа, идущих подряд. 

In [8]:
def extract_labels(item):
    raw_toks = list(tokenize(item.text))
    words = [tok.text for tok in raw_toks]
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item.text)
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    for e in item.entities:
        e_words = sorted({idx for idx in char2word[e.start:e.end] if idx is not None})
        word_labels[e_words[0]] = 'B-' + e.entity_type
        for idx in e_words[1:]:
            word_labels[idx] = 'I-' + e.entity_type

    return {'tokens': words, 'tags': word_labels}

In [9]:
print(extract_labels(drugs[101]))

{'tokens': ['нам', 'прописали', ',', 'так', 'мой', 'ребенок', 'сыпью', 'покрылся', ',', 'глаза', 'опухли', ',', 'сверху', 'и', 'снизу', 'на', 'веках', 'высыпала', 'сыпь', ',', '(', '8', 'месяцев', 'сыну', ')', 'А', 'от', 'виферона', 'такого', 'не', 'было', '...', 'У', 'кого', 'ещё', 'такие', 'побочки', ',', 'отзовитесь', '!', '1', 'Чем', 'спасались', '?'], 'tags': ['O', 'O', 'O', 'O', 'O', 'O', 'B-ADR', 'I-ADR', 'O', 'B-ADR', 'I-ADR', 'O', 'O', 'O', 'O', 'B-ADR', 'I-ADR', 'I-ADR', 'I-ADR', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Drugform', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']}


In [10]:
ner_data = [extract_labels(item) for item in drugs]
ner_train, ner_test = train_test_split(ner_data, test_size=0.1, random_state=1)

**Пример данных**

In [11]:
pd.options.display.max_colwidth = 300
pd.DataFrame(ner_train).sample(3)

,tokens,tags
1280,"[В, указанный, промежуток, времени, дочка, не, болела, никакими, вирусными, инфекциями, ,, может, это, никак, не, связано, с, приемом, Анаферона, .]","[O, O, O, O, O, O, O, O, B-Finding, I-Finding, O, O, O, O, O, O, O, O, B-Drugname, O]"
514,"[С, гостями, ,, танцами, ,, подарками, и, поздравлениями, ,, как, и, хотела, ), ), .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
2990,"[Я, думаю, ,, что, их, нужно, было, начать, принимать, немного, раньше, ,, а, не, тогда, ,, когда, уже, ребенок, прям, заболел, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"


**Соберём все виды меток в список** 

In [12]:
label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

['O',
 'B-ADR',
 'B-DI',
 'B-Drugclass',
 'B-Drugform',
 'B-Drugname',
 'B-Finding',
 'I-ADR',
 'I-DI',
 'I-Drugclass',
 'I-Drugform',
 'I-Drugname',
 'I-Finding']

Сложим наши данные в объект [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), нативный для huggingface.

In [13]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 4328
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 481
    })
})

**Предобработка данных**

Прежде чем мы сможем передать эти тексты в нашу модель, нам необходимо их предварительно обработать. Это делается с помощью Transformers `Tokenizer`, который (как следует из названия) токенизирует входные данные (включая преобразование токенов в соответствующие идентификаторы в предварительно обученном словаре) и помещает их в формат, ожидаемый моделью, а также генерирует другие входные данные, которые требуются модели.

Чтобы сделать все это, мы создаем экземпляр нашего токенизатора с помощью метода AutoTokenizer.from_pretrained, который гарантирует:

— получение токенизатора, соответствующего архитектуре модели, которую мы хотим использовать,
— загружаем словарь, используемый при предварительной тренировке этого конкретного чекпоинта.

Этот словарь будет кэширован, поэтому он не будет загружаться снова при следующем запуске ячейки.

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [15]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [2, 9944, 16, 881, 550, 835, 15503, 5, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

Если, как в данном случае, ваши входные данные уже разделены на слова, вам следует передать список слов в ваш токензиер с аргументом `is_split_into_words=True`:

In [16]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [2, 9944, 16, 881, 550, 835, 15503, 7440, 996, 6301, 18, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Преобразователи часто предварительно обучаются с помощью токенизаторов подслов, а это означает, что даже если входные данные уже были разделены на слова, каждое из этих слов может быть снова разделено токенизатором.

In [17]:
example = ner_train[5]
print(example["tokens"])

['Этим', 'средством', 'является', '"', 'Амизон', '"', ',', 'которое', 'хорошо', 'прорекламировано', ',', 'однако', 'такого', 'эффекта', 'как', 'говорят', 'нет', '.']


In [18]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'Эти', '##м', 'средство', '##м', 'является', '"', 'А', '##ми', '##зон', '"', ',', 'которое', 'хорошо', 'про', '##рек', '##лами', '##ровано', ',', 'однако', 'такого', 'э', '##ффект', '##а', 'как', 'говорят', 'нет', '.', '[SEP]']


Чтобы перейти с уровня слов на уровень subword tokens, нужно ещё раз предобработать тексты.

In [19]:
len(example["tags"]), len(tokenized_input["input_ids"])

(18, 29)

Здесь токенизатор возвращает выходные данные, содержащие метод word_ids, который может нам помочь.

In [20]:
print(tokenized_input.word_ids())

[None, 0, 0, 1, 1, 2, 3, 4, 4, 4, 5, 6, 7, 8, 9, 9, 9, 9, 10, 11, 12, 13, 13, 13, 14, 15, 16, 17, None]


Совместим метки с обработанными входными идентификаторами.

In [21]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example["tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

29 29


Применим функцию обработки образцов. Передаем их токенизатору с аргументом truncation=True (чтобы обрезать тексты, размер которых превышает максимальный размер, разрешенный моделью) и is_split_into_words=True (как показано выше). Затем мы выравниваем метки с идентификаторами токенов.

In [22]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            
            if word_idx is None:
                label_ids.append(-100)
           
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [23]:
tokenize_and_align_labels(ner_data['train'][22:23])

{'input_ids': [[2, 3130, 3374, 23324, 871, 314, 1556, 14068, 16902, 1029, 6899, 18, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]]}

Команда "map" обработает все предложения одной строкой

In [24]:
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

**Точная настройка модели**

Когда наши данные готовы, мы можем загрузить предварительно обученную модель и точно настроить ее. Поскольку все наши задачи связаны с классификацией токенов, мы используем класс AutoModelForTokenClassification. Как и в случае с токенизатором, метод from_pretrained загрузит и кэширует для нас модель. Единственное, что нам нужно указать, это количество меток для нашей проблемы (которое мы можем получить из функций, как было показано ранее)

In [25]:
label_list

['O',
 'B-ADR',
 'B-DI',
 'B-Drugclass',
 'B-Drugform',
 'B-Drugname',
 'B-Finding',
 'I-ADR',
 'I-DI',
 'I-Drugclass',
 'I-Drugform',
 'I-Drugname',
 'I-Finding']

In [26]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
)

Здесь мы устанавливаем оценку, которая будет выполняться в конце каждой эпохи, настраиваем скорость обучения, используем параметр «batch_size», определенный в верхней части блокнота, и настраиваем количество эпох для обучения, а также затухание весов

Применим сортировку данных, которая будет группировать наши обработанные примеры вместе, применяя заполнение, чтобы сделать их всех одинакового размера (каждое дополнение будет дополнено до длины самого длинного примера). Для этой задачи в библиотеке Transformers есть средство сортировки данных, которое не только дополняет входные данные, но и метки

In [28]:
data_collator = DataCollatorForTokenClassification(tokenizer)

Загрузим метрику seqeval (которая обычно используется для оценки результатов в наборе данных CONLL) через библиотеку наборов данных.

In [29]:
metric = load_metric("seqeval")

In [30]:
example = ner_train[4]
labels = example['tags']
metric.compute(predictions=[labels], references=[labels])

{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 1.0}

Следующая функция выполняет всю постобработку результата Trainer.evaluate (который представляет собой именованный кортеж, содержащий прогнозы и метки) перед применением метрики

In [31]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [32]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [33]:
trainer.evaluate()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 2.770181655883789,
 'eval_precision': 0.01888934764657308,
 'eval_recall': 0.11144945188794153,
 'eval_f1': 0.032303618711385707,
 'eval_accuracy': 0.0321798729521899,
 'eval_runtime': 8.5183,
 'eval_samples_per_second': 56.467,
 'eval_steps_per_second': 1.878}

В начале обучения заморозим все параметры в модели, кроме последнего слоя, и посмотрим, насколько хорошо она обучится.

In [34]:
for param in model.bert.parameters():
    param.requires_grad = False

In [35]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)
        print(param)

classifier.weight
Parameter containing:
tensor([[ 0.0369, -0.0169,  0.0101,  ...,  0.0295, -0.0226,  0.0197],
        [-0.0258, -0.0180, -0.0017,  ...,  0.0095,  0.0075,  0.0287],
        [-0.0171, -0.0056,  0.0061,  ..., -0.0252,  0.0484,  0.0083],
        ...,
        [ 0.0155, -0.0223, -0.0079,  ..., -0.0174, -0.0795, -0.0046],
        [ 0.0016, -0.0109,  0.0151,  ...,  0.0001,  0.0033,  0.0244],
        [-0.0341, -0.0089,  0.0606,  ..., -0.0234, -0.0175,  0.0143]],
       device='cuda:0', requires_grad=True)
classifier.bias
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0',
       requires_grad=True)


In [36]:
noisy_logger.setLevel(logging.WARNING)

In [37]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,2.429248,0.021914,0.104750,0.036245,0.190488
2,No log,2.140264,0.024432,0.060901,0.034874,0.501254
3,No log,1.901813,0.034049,0.028015,0.030738,0.717987
4,2.212500,1.710953,0.060759,0.014616,0.023564,0.791708
5,2.212500,1.562821,0.122951,0.009135,0.017007,0.811601
6,2.212500,1.451857,0.081633,0.002436,0.004731,0.815530
7,2.212500,1.372272,0.083333,0.001218,0.002401,0.816867
8,1.497800,1.319215,0.066667,0.000609,0.001207,0.817369
9,1.497800,1.288893,0.083333,0.000609,0.001209,0.817536
10,1.497800,1.279003,0.111111,0.000609,0.001211,0.817787


TrainOutput(global_step=1360, training_loss=1.6999690336339615, metrics={'train_runtime': 60.3185, 'train_samples_per_second': 717.524, 'train_steps_per_second': 22.547, 'total_flos': 48055430014560.0, 'train_loss': 1.6999690336339615, 'epoch': 10.0})

Модель недообучилась: похоже, что нужно обучить больше слоёв. Разморозим их все (но, воможно, более правильно было бы разморозить лишь несколько верхних), и поучимся ещё эпох 20.

In [38]:
# разморозка
for param in model.parameters():
    param.requires_grad = True

In [39]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
)

In [40]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [41]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.717055,0.820513,0.038977,0.074419,0.822885
2,No log,0.629035,0.573109,0.207674,0.304873,0.841107
3,No log,0.569805,0.573317,0.290499,0.385610,0.851387
4,0.615300,0.533208,0.556279,0.364190,0.440191,0.859746
5,0.615300,0.501788,0.528667,0.432400,0.475712,0.865095
6,0.615300,0.480193,0.560831,0.460414,0.505686,0.870528
7,0.615300,0.464430,0.564413,0.482948,0.520512,0.873036
8,0.428100,0.453347,0.589091,0.493301,0.536957,0.876463
9,0.428100,0.441357,0.573549,0.517661,0.544174,0.878051
10,0.428100,0.431835,0.590631,0.529842,0.558587,0.880726


TrainOutput(global_step=2720, training_loss=0.40825541440178365, metrics={'train_runtime': 150.6016, 'train_samples_per_second': 574.761, 'train_steps_per_second': 18.061, 'total_flos': 95941710795840.0, 'train_loss': 0.40825541440178365, 'epoch': 20.0})

In [42]:
trainer.evaluate()

{'eval_loss': 0.3958091139793396,
 'eval_precision': 0.5976070528967254,
 'eval_recall': 0.5779537149817296,
 'eval_f1': 0.5876160990712075,
 'eval_accuracy': 0.8874122367101304,
 'eval_runtime': 0.9065,
 'eval_samples_per_second': 530.639,
 'eval_steps_per_second': 17.651,
 'epoch': 20.0}

In [43]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'ADR': {'precision': 0.35353535353535354,
  'recall': 0.1320754716981132,
  'f1': 0.19230769230769232,
  'number': 265},
 'DI': {'precision': 0.3864353312302839,
  'recall': 0.5372807017543859,
  'f1': 0.44954128440366964,
  'number': 456},
 'Drugclass': {'precision': 0.8187919463087249,
  'recall': 0.7577639751552795,
  'f1': 0.7870967741935484,
  'number': 161},
 'Drugform': {'precision': 0.8204081632653061,
  'recall': 0.7528089887640449,
  'f1': 0.78515625,
  'number': 267},
 'Drugname': {'precision': 0.7505422993492408,
  'recall': 0.8628428927680798,
  'f1': 0.8027842227378189,
  'number': 401},
 'Finding': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 92},
 'overall_precision': 0.5976070528967254,
 'overall_recall': 0.5779537149817296,
 'overall_f1': 0.5876160990712075,
 'overall_accuracy': 0.8874122367101304}

In [44]:
cm = pd.DataFrame(
    confusion_matrix(sum(true_labels, []), sum(true_predictions, []), labels=label_list),
    index=label_list,
    columns=label_list
)
cm

,O,B-ADR,B-DI,B-Drugclass,B-Drugform,B-Drugname,B-Finding,I-ADR,I-DI,I-Drugclass,I-Drugform,I-Drugname,I-Finding
O,9621,8,75,8,21,52,0,1,5,0,0,0,0
B-ADR,101,44,115,4,1,0,0,0,0,0,0,0,0
B-DI,168,1,265,3,12,6,0,0,1,0,0,0,0
B-Drugclass,27,1,6,122,0,5,0,0,0,0,0,0,0
B-Drugform,58,0,1,0,202,6,0,0,0,0,0,0,0
B-Drugname,22,0,15,3,6,355,0,0,0,0,0,0,0
B-Finding,25,12,48,1,2,3,0,0,1,0,0,0,0
I-ADR,134,21,34,0,1,2,0,2,8,0,0,0,0
I-DI,171,8,61,3,0,3,0,0,6,0,0,0,0
I-Drugclass,0,0,0,0,0,0,0,0,0,0,0,0,0


In [45]:
model.save_pretrained('ner_bert.bin')
tokenizer.save_pretrained('ner_bert.bin')

('ner_bert.bin/tokenizer_config.json',
 'ner_bert.bin/special_tokens_map.json',
 'ner_bert.bin/vocab.txt',
 'ner_bert.bin/added_tokens.json',
 'ner_bert.bin/tokenizer.json')

#### Применение модели

In [46]:
text = ' '.join(ner_test[10]['tokens'])
text

'К сожалению хочу сказать , что при действительно серьезных проблемах с нервной системой она оказалась бесполезной , все равно что " мертвому припарка " .'

In [47]:
tokens = tokenizer(text, return_tensors='pt')
tokens = {k: v.to(model.device) for k, v in tokens.items()}

with torch.no_grad():
    pred = model(**tokens)
pred.logits.shape

torch.Size([1, 52, 13])

In [48]:
indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
token_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
for t, idx in zip(token_text, indices):
    print(f'{t:15s} {label_list[idx]:10s}')

[CLS]           O         
К               O         
со              O         
##жал           O         
##ению          O         
х               O         
##очу           O         
с               O         
##каз           O         
##ать           O         
,               O         
что             O         
при             O         
дей             O         
##ств           O         
##итель         O         
##но            O         
се              O         
##рь            O         
##ез            O         
##ных           O         
проблема        O         
##х             O         
с               O         
не              B-DI      
##рв            B-DI      
##ной           B-DI      
систем          O         
##ой            O         
она             O         
оказалась       O         
б               O         
##ес            O         
##по            O         
##ле            O         
##зно           O         
##й             O         
,

Более простое применение модели: пайплайн от huggingface

In [49]:
pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device=0)

In [50]:
print(text)
print(pipe(text))

К сожалению хочу сказать , что при действительно серьезных проблемах с нервной системой она оказалась бесполезной , все равно что " мертвому припарка " .
[{'entity_group': 'DI', 'score': 0.32064527, 'word': 'нервной', 'start': 71, 'end': 78}]
